In [ ]:
from typing import Dict, List, Any
import pandas as pd

from doraemon import Doraemon
from sketch_of_thought import SoT
from reasoning_path_builder import ReasoningPath
from prompt_template_collection import PromptTemplate

df=pd.read_pickle('/kaggle/input/building-hotpotqa-injected-shuffled-dataset/hotpotqa_injected_shuffled_all_in_one_df.pkl')
df.shape

In [2]:
logger=Doraemon.get_logger(name=__name__, logfile='sot_on_hotpotqa_injected')

df.rename(columns={'answer':'ground_truth'}, inplace=True)


df['evidence_injected'] = df['evidence_injected'].apply(lambda x: ' '.join(x))
df['evidence_type']='Injected'

In [ ]:
df['question'] = df.apply(lambda row: f"{row['question']} \nContext: {row['evidence_injected']}", axis=1)
logger.info(df.iloc[0].question)

In [4]:
def final_prompt(question:str) -> str:
    return SoT.few_short_support(
        task_name='HOTPOTQA', 
        prompt=SoT.get_initialized_prompt(paradigm='conceptual_chaining'),
        few_short=SoT.get_few_short(task_name='HOTPOTQA'),
        question=question)

df['prompt']=df['question'].apply(final_prompt)

In [ ]:
tasks: List[Dict[str, Any]]=df.head(1200).to_dict(orient='records')

temperatures = [i * 0.25 for i in range(9)]

final_tasks=[]
for t in tasks:
    for tp in temperatures:
        final_tasks.append((t, tp))

logger.info(len(final_tasks))

In [ ]:
D:Dict[str, str] = PromptTemplate.sot_template_hotpotqa()
logger.info(len(D))

results:List[Dict]=ReasoningPath.run_parallel_inference(final_tasks, D, logger, 'sots', None)

In [ ]:
logger.info(results[0])

In [8]:
filtered_results = [r for r in results if r is not None]

pd.DataFrame(filtered_results).to_pickle('sots_df.pkl')